# Multi-Turn Chat Benchmark

```bash
gcloud container clusters create-auto cluster-1 \
    --location=us-central1

helm repo add kubeai https://www.kubeai.org
helm repo update
curl -L -O https://raw.githubusercontent.com/substratusai/kubeai/refs/heads/main/charts/kubeai/values-gke.yaml
helm upgrade --install kubeai kubeai/kubeai \
    -f values-gke.yaml \
    --set secrets.huggingface.token=$HUGGING_FACE_HUB_TOKEN \
    --set metrics.prometheusOperator.vLLMPodMonitor.enabled=true \
    --set open-webui.enabled=false \
    --wait
```

In [ ]:
from subprocess import run, PIPE
import json
from kubernetes import config, dynamic
from kubernetes.client import api_client
from copy import deepcopy

In [ ]:
k8s_client = dynamic.DynamicClient(
    api_client.ApiClient(configuration=config.load_kube_config())
)
models_client = k8s_client.resources.get(api_version="kubeai.org/v1", kind="Model")

In [114]:
#base_model = {
#    "apiVersion": "kubeai.org/v1",
#    "kind": "Model",
#    "metadata": {
#        "name": "bench",
#        "namespace": "default",
#    },
#    "spec": {
#        "features": ["TextGeneration"],
#        "url": "ollama://qwen2:0.5b",
#        "engine": "OLlama",
#        "resourceProfile": "cpu:2"
#    },
#}
base_model = {
    "apiVersion": "kubeai.org/v1",
    "kind": "Model",
    "metadata": {
        "name": "bench",
        "namespace": "default",
    },
    "spec": {
        "features": ["TextGeneration"],
        "url": "hf://neuralmagic/Meta-Llama-3.1-8B-Instruct-FP8",
        "engine": "VLLM",
        "args": [
            "--enable-prefix-caching",
            "--max-model-len=16384",
            "--max-num-batched-token=16384",
            "--gpu-memory-utilization=0.8",
            "--disable-log-requests"
        ],
        "resourceProfile": "nvidia-gpu-l4:1"
    },

}

In [166]:
benches = [
    {
        "thread_count": 2000,
        "max_concurrent_threads": 300,
    },
]
specs = [
    {
        "minReplicas": 2,
        "maxReplicas": 2,
        "loadBalancing": {
            "strategy": "PrefixHash",
            "prefixHash": {
                "meanLoadFactor": 115,
                "prefixCharLength": 100,
                "replication": 1000,
            },
        },
    },
    {
        "minReplicas": 2,
        "maxReplicas": 2,
        "loadBalancing": {
            "strategy": "LeastLoad",
            "prefixHash": {
                "meanLoadFactor": 115,
                "prefixCharLength": 100,
                "replication": 1000,
            },
        },
    },
] 


In [167]:
all_results = []

i = 0
for spec in specs:
    for  bench in benches:
        print(bench)
        try:
            # Start a fresh instance of the benchmark Pod.
            !kubectl apply -f ./bench-pod.yaml
            !kubectl wait --timeout 10m --for=condition=Ready -f ./bench-pod.yaml

            model = deepcopy(base_model)
            model["spec"].update(spec)
            #model["metadata"]["name"] = model["metadata"]["name"] + f'-{i}'
            #models_client.create(body=model)
            models_client.patch(
                body=model,
                content_type="application/apply-patch+yaml",
                field_manager="benchmark",
            )


            model_name = model.get("metadata").get("name")
            model_replicas = model.get("spec").get("minReplicas")

            !kubectl wait --timeout 30m --for=jsonpath='.status.replicas.ready'={model_replicas} model/{model_name}

            thread_count = bench.get("thread_count")
            max_concurrent_threads = bench.get("max_concurrent_threads")
            cmd = f'kubectl exec bench -- bench --threads=./data/large-exact.json --thread-count={thread_count} --max-concurrent-threads={max_concurrent_threads} --request-model={model_name} --max-completion-tokens=40 --request-timeout=2m --seed=2 --format=json'
            print(cmd)

            output = run(cmd, shell=True, stdout=PIPE, encoding='utf8')
            result = json.loads(output.stdout)
            print(result)
            all_results.append({
                "spec": spec,
                "bench": bench,
                "result": result
            }) 
        finally:
            models_client.delete(name=model_name, namespace="default")
            !kubectl delete --now -f ./bench-pod.yaml
            i+=1

all_results

{'thread_count': 2000, 'max_concurrent_threads': 300}
pod/bench created
pod/bench condition met
model.kubeai.org/bench condition met
kubectl exec bench -- bench --threads=./data/large-exact.json --thread-count=2000 --max-concurrent-threads=300 --request-model=bench --max-completion-tokens=40 --request-timeout=2m --seed=2 --format=json


2025/02/24 03:29:15 Shuffling dataset threads
2025/02/24 03:29:15 Trimming dataset threads (9204) to specified thread count (2000)
2025/02/24 03:29:15 Starting run...
2025/02/24 03:36:18 Run completed, starting summarization...


{'input_thread_count': 2000, 'input_messages_per_thread': {'mean': 7.441}, 'duration': '7m3.113770224s', 'request_count': 14882, 'request_duration': {'mean': '7.689761059s'}, 'chunks_per_request': {'mean': 38.956255879586074}, 'failed_threads': 0, 'run_output_throughput': 1370.4540972349312, 'run_total_throughput': 15091.186932109475, 'ttft': {'mean': '2.133812223s'}, 'itl': {'mean': '141.652458ms'}, 'prompt_tokens': 5805431, 'cached_prompt_tokens': 0, 'completion_tokens': 579858, 'total_tokens': 6385289}
pod "bench" deleted
{'thread_count': 2000, 'max_concurrent_threads': 300}
pod/bench created
pod/bench condition met
model.kubeai.org/bench condition met
kubectl exec bench -- bench --threads=./data/large-exact.json --thread-count=2000 --max-concurrent-threads=300 --request-model=bench --max-completion-tokens=40 --request-timeout=2m --seed=2 --format=json


2025/02/24 03:38:05 Shuffling dataset threads
2025/02/24 03:38:05 Trimming dataset threads (9204) to specified thread count (2000)
2025/02/24 03:38:05 Starting run...
2025/02/24 03:44:45 Run completed, starting summarization...


{'input_thread_count': 2000, 'input_messages_per_thread': {'mean': 7.441}, 'duration': '6m39.71940859s', 'request_count': 14882, 'request_duration': {'mean': '7.317937037s'}, 'chunks_per_request': {'mean': 38.85230479774224}, 'failed_threads': 0, 'run_output_throughput': 1446.6898218423585, 'run_total_throughput': 15945.045106722522, 'ttft': {'mean': '1.790251177s'}, 'itl': {'mean': '140.861199ms'}, 'prompt_tokens': 5795274, 'cached_prompt_tokens': 0, 'completion_tokens': 578270, 'total_tokens': 6373544}
pod "bench" deleted


[{'spec': {'minReplicas': 2,
   'maxReplicas': 2,
   'loadBalancing': {'strategy': 'PrefixHash',
    'prefixHash': {'meanLoadFactor': 115,
     'prefixCharLength': 100,
     'replication': 1000}}},
  'bench': {'thread_count': 2000, 'max_concurrent_threads': 300},
  'result': {'input_thread_count': 2000,
   'input_messages_per_thread': {'mean': 7.441},
   'duration': '7m3.113770224s',
   'request_count': 14882,
   'request_duration': {'mean': '7.689761059s'},
   'chunks_per_request': {'mean': 38.956255879586074},
   'failed_threads': 0,
   'run_output_throughput': 1370.4540972349312,
   'run_total_throughput': 15091.186932109475,
   'ttft': {'mean': '2.133812223s'},
   'itl': {'mean': '141.652458ms'},
   'prompt_tokens': 5805431,
   'cached_prompt_tokens': 0,
   'completion_tokens': 579858,
   'total_tokens': 6385289}},
 {'spec': {'minReplicas': 2,
   'maxReplicas': 2,
   'loadBalancing': {'strategy': 'LeastLoad',
    'prefixHash': {'meanLoadFactor': 115,
     'prefixCharLength': 100,
 

In [168]:
for r in all_results:
    print(f'Strategy {r["spec"]["loadBalancing"]["strategy"]}: TTFT={r["result"]["ttft"]["mean"]} ITL={r["result"]["itl"]["mean"]} TPS(total)={r["result"]["run_total_throughput"]}')

Strategy PrefixHash: TTFT=2.133812223s ITL=141.652458ms TPS(total)=15091.186932109475
Strategy LeastLoad: TTFT=1.790251177s ITL=140.861199ms TPS(total)=15945.045106722522
